# 1. Web Scrape

In [1]:
#import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import string

Matplotlib is building the font cache using fc-list. This may take a moment.


In [73]:
from abc import ABC, abstractmethod

class ScrapeDebate(ABC):
    
    def __init__(self):
        pass
    
    @abstractmethod
    def process_paragraph(self, p):
        pass
    
    @abstractmethod
    def scrape_data(self, url, start, end):
        pass

In [74]:
class ScrapeScribie(ScrapeDebate):
    def process_paragraph(self, p):
        try:
            x,y = list(p.children)
            time = re.findall('^(\d:\d{2}:\d{2})', x.b.text)[0]
            speaker = re.findall('^\d:\d{2}:\d{2} (.*):', x.b.text)[0]
            body = y.strip()

        except:
            speaker = None
            time = None
            body = None

        return {'speaker': speaker, 'time': time, 'body': body}        
    
    
    def scrape_data(self, url, start, end):
        try:
            bs_obj = BeautifulSoup(urlopen(url).read())
            for a in bs_obj.findAll('a'):
                del a['href']
            #paragraphs = [p.text for p in bs_obj.findAll('p')[start:end]]
            data = pd.DataFrame([self.process_paragraph(p) for p in bs_obj.findAll('p')[start:end]])
            return data.dropna(how='all')

        except:
            print ('scrape failed!')

In [157]:
class ScrapeRev(ScrapeDebate):
    def process_paragraph(self, p):
        try:
            speaker = re.match('(.*): \(', p).groups()[0]
        except:
            speaker = None
            
        try:
            time = re.findall('\(.*\)', p)[0].replace('(', '').replace(')', '')
        except:
            time = None
            
        try:
            body = p.split(')')[1].strip()        
        except:
            body = None

        return {'speaker': speaker, 'time': time, 'body': body}        
    
    def scrape_data(self, url, start, end):
        try:
            bs_obj = BeautifulSoup(urlopen(url).read())
            for a in bs_obj.findAll('a'):
                del a['href']
            paragraphs = [p.text for p in bs_obj.findAll('p')[start:end]]
            data = pd.DataFrame([self.process_paragraph(p) for p in paragraphs])
            return data.dropna(how='all')

        except:
            print ('scrape failed!')

In [160]:
scribie = ScrapeScribie()
rev = ScrapeRev()

In [197]:
url1_1 = 'https://scribie.com/blog/2019/07/full-democratic-presidential-debate-night-one-july-30-transcripts2020/'
url1_2 = 'https://www.rev.com/blog/transcript-from-night-2-of-the-2019-democratic-debates'
url2_1 = 'https://www.rev.com/blog/transcript-of-july-democratic-debate-night-1-full-transcript-july-30-2019'
url2_2 = 'https://www.rev.com/blog/transcripts/transcript-of-july-democratic-debate-2nd-round-night-2-full-transcript-july-31-2019'
url3 = 'https://scribie.com/blog/2019/09/transcript-of-3rd-democratic-debate-at-tsu-sep-12-2019-transcripts2020/'
url4 = 'https://scribie.com/blog/2019/10/fourth-democratic-presidential-debate-ohio-10-15-19-transcripts2020/'
url5 = 'https://scribie.com/blog/2019/11/fifth-democratic-presidential-debate-11-20-19-transcripts2020/'
url6 = 'https://www.rev.com/blog/december-democratic-debate-transcript-sixth-debate-from-los-angeles'
url7 = 'https://www.rev.com/blog/transcripts/january-iowa-democratic-debate-transcript'

In [173]:
data1_1 = scribie.scrape_data(url1_1, 2, -1)
data1_2 = rev.scrape_data(url1_2, 2, -2)
data2_1 = rev.scrape_data(url2_1, 2, -11)
data2_2 = rev.scrape_data(url2_2, 2, -2)

In [174]:
data3 = scribie.scrape_data(url3, 2, -11)
data4 = scribie.scrape_data(url4, 2, -11)
data5 = scribie.scrape_data(url5, 2, -11)
data6 = rev.scrape_data(url6, 3, -2)

In [194]:
data7 = rev.scrape_data(url7, 2, -2)
data7.loc[0, 'body'] = 'All right, so let’s begin right now. Just this month, the United States and Iran were on the brink of war, which has reignited the debate over America’s role in the world and which of you is best prepared to be commander-in-chief. So let’s have the debate right now. Senator Sanders, why are you best prepared — the best prepared person on this stage to be commander-in-chief?'
data7 = data7.reset_index().drop('index', axis=1)